In [1]:
# path variables
import sys
import json
project_path = '/Users/naresh/Downloads/ds_models/onboarding_fraud_model_v2/'
sys.path.insert(0, project_path+'config')

# core libraries
import datetime
import pandas as pd
import numpy as np
import warnings

from config import SQLQuery
from collections import Counter

warnings.filterwarnings("ignore")

In [2]:
q = SQLQuery('snowflake')

#### Train & Test Dataset

In [3]:
year = '2021_2022'

In [4]:
# query to fetch the required data, variables to change are the from_date, to_date, od_amount, od_count and days_on_platform
df_raw = q(""" 

-- Data extracting query
with all_businesses as (
select a.application_id, business_id, expensed_fraud_loss, business_type,
case when closure_reason ilike '%system closeout%' then 'System Closeout' else closure_reason end as closure_reason, 
account_create_date, account_close_date, closure_requested_by, closure_is_suspect_fraud,
ACCOUNT_FUNDED_AT, datediff('day', account_create_date, coalesce(ACCOUNT_FUNDED_AT, current_date())) as days_to_fund_account, first_credit_amount,
datediff('day', account_create_date, coalesce(account_close_date, current_date())) as days_to_close_fraud_account,
b.fraud_score, b.deposit_score,
case 
    when b.fraud_score<=0.35 then 1
    when b.fraud_score<=0.6 then 2
    else 3
end as fraud_bin,
case 
    when b.fraud_score<=0.35 then 'low'
    when b.fraud_score<=0.6 then 'moderate'
    else 'high'
end as fraud_category
from prod_db.data.businesses a
left join prod_db.models.onboarding_model_results b on a.application_id=b.application_id
where 1=1 
and account_create_date between '2021-01-01' and '2022-12-31'
and business_type ilike '%sole%'
)

-- Pulling transactions data
,txns_data as (
select a.application_id, a.business_id, a.business_type, a.account_create_date, a.account_funded_at, a.days_to_close_fraud_account,
a.days_to_fund_account, a.account_close_date, b.transaction_id, b.amount, b.type, b.medium, b.status, b.transaction_date, b.running_balance
from all_businesses a
left join prod_db.data.TRANSACTIONS b on a.business_id=b.business_id
)

-- Extracting OD accounts info and filtering for 90 days on book data
,full_txns_data as (
select business_id, 
sum(case when account_funded_at is not null then 1 else 0 end) as account_funded,
sum(case when status='active' and running_balance < -250 then 1 else 0 end) as overdraft_count
from txns_data 
where 1=1 
and datediff('day', account_create_date, transaction_date) <= 90 
and days_to_close_fraud_account <= 90 
and account_close_date is not null
group by 1
)

-- Filtering OD accounts info using the frequency
,overdraft_apps_data as (
select extract('year', a.account_create_date) as year, count(distinct a.business_id) as closed_apps_by_novo_90_days_overdraft_apps 
from all_businesses a 
inner join full_txns_data b on a.business_id=b.business_id
where b.overdraft_count >= 2
group by 1
)

-- Tagging the final target
,final_target_tmp as (
select distinct extract('year', a.account_create_date) as year, a.business_id from all_businesses a
inner join full_txns_data b on a.business_id=b.business_id where 1=1 and b.overdraft_count >= 2

union

select distinct extract('year', a.account_create_date) as year, a.business_id from all_businesses a 
where 1=1 and a.account_close_date is not null and a.closure_requested_by not ilike 'user' and a.expensed_fraud_loss>0
)

-- Filtering the final target
,final_target_tmp2 as (select year, business_id, rank() over(partition by business_id order by year) as rk from final_target_tmp order by rk)


,final_target_tmp3 as ( 
select a.application_id, a.business_id, 
case when a.business_id=b.business_id then 1 else 0 end as target
from all_businesses a
left join 
(select * from final_target_tmp2 where 1=1 and rk=1) b on a.business_id=b.business_id
)

-- final_target data
,final_target as ( 
select a.application_id, a.business_id, a.target, coalesce(b.expensed_fraud_loss,0) as expensed_fraud_loss
from final_target_tmp3 a
left join 
prod_db.data.businesses b on a.business_id=b.business_id
)

-- select count(*) as cnt from all_businesses
-- where 1=1
-- and ACCOUNT_FUNDED_AT is not null
-- and days_to_fund_account <= 90

,apps_data as (
select distinct a.application_id,
a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
a.email, a.email_domain, a.business_pitch,
a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
a.number_of_employees, a.purpose_of_account, a.has_international_business,
a.industry_name, a.industry_category_name, a.website, 
a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
a.company_name, a.phone, a.device, a.flow
from prod_db.data.applications a
inner join final_target b on a.application_id=b.application_id
)

,alloy_persons_data_tmp as (
select distinct
d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
rank() over(partition by d.APPLICATION_ID order by d.APPLICATION_VERSION_ID desc) as ranking
from prod_db.data.ALLOY_EVALUATIONS_PERSONS d
inner join final_target b on d.application_id=b.application_id
),


alloy_persons_data_tmp2 as (
select * from alloy_persons_data_tmp where ranking=1
),

alloy_persons_data as (
select distinct
d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
d.sentilink_id_theft_score, d.socure_reason_code, d.socure_kyc_field_validations, 
d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
first_value(d.socure_sigma) over(partition by d.application_id order by d.socure_sigma desc) as socure_sigma
from alloy_persons_data_tmp2 d
)

select 
c.*,
d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,

a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
a.email, a.email_domain, a.business_pitch,
a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
a.number_of_employees, a.purpose_of_account, a.has_international_business,
a.industry_name, a.industry_category_name, a.website, 
a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
a.company_name, a.phone, a.device, a.flow, e.fraud_score, e.deposit_score

from final_target c
inner join alloy_persons_data d on c.application_id=d.application_id
inner join apps_data a on c.application_id=a.application_id
left join prod_db.models.onboarding_model_results e on c.application_id=e.application_id

    """)


In [5]:
df_raw.shape

(65769, 63)

In [6]:
# drop null application ids
df_raw = df_raw.dropna(subset='application_id')

# get non-ns vs ns accounts split
df_raw['target'].value_counts(dropna=False)

0    63607
1     2162
Name: target, dtype: int64

In [7]:
# drop duplicate columns
df_raw = df_raw.loc[:,~df_raw.columns.duplicated()].copy()
df_raw.shape

(65769, 63)

In [8]:
print('Applications with $0 fraud loss:', df_raw[(df_raw['expensed_fraud_loss']==0) & (df_raw['target']==1)].shape[0])
print('Applications with fraud loss greater than $0 :', df_raw[(df_raw['expensed_fraud_loss']>0) & (df_raw['target']==1)].shape[0])
print('Applications with $500 fraud loss or more:', df_raw[(df_raw['expensed_fraud_loss']>=500) & (df_raw['target']==1)].shape[0])
print('Applications with $1K fraud loss or more:', df_raw[(df_raw['expensed_fraud_loss']>=1000) & (df_raw['target']==1)].shape[0])
print('Applications with $5K fraud loss or more:', df_raw[(df_raw['expensed_fraud_loss']>=5000) & (df_raw['target']==1)].shape[0])
print('Applications with $10K fraud loss or more:', df_raw[(df_raw['expensed_fraud_loss']>=10000) & (df_raw['target']==1)].shape[0])
print('Applications with $20K fraud loss or more:', df_raw[(df_raw['expensed_fraud_loss']>=20000) & (df_raw['target']==1)].shape[0])


Applications with $0 fraud loss: 190
Applications with fraud loss greater than $0 : 1972
Applications with $500 fraud loss or more: 1029
Applications with $1K fraud loss or more: 737
Applications with $5K fraud loss or more: 79
Applications with $10K fraud loss or more: 17
Applications with $20K fraud loss or more: 5


In [9]:
# pd.read_pickle(project_path+'data/onboarding_fraud_v2_raw_dataset_'+year+'_2024-06-06.pkl').shape

In [10]:
# save the dataset
# file = 'onboarding_fraud_v2_raw_dataset_'+year+'_' + str(datetime.date.today()) + '.pkl'
file = 'onboarding_fraud_v2_raw_dataset_'+year+'_2024-06-06.pkl'
path = project_path + 'data/'
df_raw.reset_index(inplace=True)
df_raw.to_pickle(path+file)

### All Applications Data

#### 2021 All Apps Data

In [4]:
# # query to fetch the required data
# df_apps = q(""" 

# with apps_data as (
# select distinct a.application_id,
# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status
# from prod_db.data.applications a
# where date(application_complete_datetime) between '2021-01-01' and '2021-12-31' and business_type ilike '%sole%'
# )

# ,alloy_persons_data_tmp as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# rank() over(partition by d.APPLICATION_ID order by d.APPLICATION_VERSION_ID desc) as ranking
# from prod_db.data.ALLOY_EVALUATIONS_PERSONS d
# inner join apps_data b on d.application_id=b.application_id
# ),


# alloy_persons_data_tmp2 as (
# select * from alloy_persons_data_tmp where ranking=1
# ),

# alloy_persons_data as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# first_value(d.socure_sigma) over(partition by d.application_id order by d.socure_sigma desc) as socure_sigma
# from alloy_persons_data_tmp2 d
# )

# select 
# d.application_id,
# d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,

# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status, e.fraud_score, e.deposit_score

# from alloy_persons_data d 
# inner join apps_data a on d.application_id=a.application_id
# left join prod_db.models.onboarding_model_results e on d.application_id=e.application_id

#     """)


In [5]:
# # save the dataset
# file = 'apps_raw_dataset_2021.pkl'
# path = project_path + 'data/'
# df_apps.reset_index(inplace=True)
# df_apps.to_pickle(path+file)

#### 2022 All Apps Data

In [4]:
# # query to fetch the required data
# df_apps = q(""" 

# with apps_data as (
# select distinct a.application_id,
# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status
# from prod_db.data.applications a
# where date(application_complete_datetime) between '2022-01-01' and '2022-12-31' and business_type ilike '%sole%'
# )

# ,alloy_persons_data_tmp as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# rank() over(partition by d.APPLICATION_ID order by d.APPLICATION_VERSION_ID desc) as ranking
# from prod_db.data.ALLOY_EVALUATIONS_PERSONS d
# inner join apps_data b on d.application_id=b.application_id
# ),


# alloy_persons_data_tmp2 as (
# select * from alloy_persons_data_tmp where ranking=1
# ),

# alloy_persons_data as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# first_value(d.socure_sigma) over(partition by d.application_id order by d.socure_sigma desc) as socure_sigma
# from alloy_persons_data_tmp2 d
# )

# select 
# d.application_id,
# d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,

# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status, e.fraud_score, e.deposit_score

# from alloy_persons_data d 
# inner join apps_data a on d.application_id=a.application_id
# left join prod_db.models.onboarding_model_results e on d.application_id=e.application_id

#     """)


In [5]:
# # save the dataset
# file = 'apps_raw_dataset_2022.pkl'
# path = project_path + 'data/'
# df_apps.reset_index(inplace=True)
# df_apps.to_pickle(path+file)

#### 2023 All Apps Data

In [6]:
# # query to fetch the required data
# df_apps = q(""" 

# with apps_data as (
# select distinct a.application_id,
# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status
# from prod_db.data.applications a
# where date(application_complete_datetime) between '2023-01-01' and '2023-12-31' and business_type ilike '%sole%'
# )

# ,alloy_persons_data_tmp as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# rank() over(partition by d.APPLICATION_ID order by d.APPLICATION_VERSION_ID desc) as ranking
# from prod_db.data.ALLOY_EVALUATIONS_PERSONS d
# inner join apps_data b on d.application_id=b.application_id
# ),


# alloy_persons_data_tmp2 as (
# select * from alloy_persons_data_tmp where ranking=1
# ),

# alloy_persons_data as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# first_value(d.socure_sigma) over(partition by d.application_id order by d.socure_sigma desc) as socure_sigma
# from alloy_persons_data_tmp2 d
# )

# select 
# d.application_id,
# d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,

# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status, e.fraud_score, e.deposit_score

# from alloy_persons_data d 
# inner join apps_data a on d.application_id=a.application_id
# left join prod_db.models.onboarding_model_results e on d.application_id=e.application_id

#     """)


In [7]:
# # save the dataset
# file = 'apps_raw_dataset_2023.pkl'
# path = project_path + 'data/'
# df_apps.reset_index(inplace=True)
# df_apps.to_pickle(path+file)

#### 2021 to 2022: All Apps Data

In [8]:
# query to fetch the required data
df_apps = q(""" 

with apps_data as (
select distinct a.application_id,
a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
a.email, a.email_domain, a.business_pitch,
a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
a.number_of_employees, a.purpose_of_account, a.has_international_business,
a.industry_name, a.industry_category_name, a.website, 
a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
a.company_name, a.phone, a.device, a.flow, a.status
from prod_db.data.applications a
where date(application_complete_datetime) between '2021-01-01' and '2022-12-31' and business_type ilike '%sole%'
)

,alloy_persons_data_tmp as (
select distinct
d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
rank() over(partition by d.APPLICATION_ID order by d.APPLICATION_VERSION_ID desc) as ranking
from prod_db.data.ALLOY_EVALUATIONS_PERSONS d
inner join apps_data b on d.application_id=b.application_id
),


alloy_persons_data_tmp2 as (
select * from alloy_persons_data_tmp where ranking=1
),

alloy_persons_data as (
select distinct
d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
d.sentilink_id_theft_score, d.socure_reason_code, d.socure_kyc_field_validations, 
d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
first_value(d.socure_sigma) over(partition by d.application_id order by d.socure_sigma desc) as socure_sigma
from alloy_persons_data_tmp2 d
)

select 
d.application_id,
d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,

a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
a.email, a.email_domain, a.business_pitch,
a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
a.number_of_employees, a.purpose_of_account, a.has_international_business,
a.industry_name, a.industry_category_name, a.website, 
a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
a.company_name, a.phone, a.device, a.flow, a.status, e.fraud_score, e.deposit_score

from alloy_persons_data d 
inner join apps_data a on d.application_id=a.application_id
left join prod_db.models.onboarding_model_results e on d.application_id=e.application_id

    """)

In [9]:
# save the dataset
file = 'apps_raw_dataset_2021_2022.pkl'
path = project_path + 'data/'
df_apps.reset_index(inplace=True)
df_apps.to_pickle(path+file)

#### 2021 to 2023: All Apps Data

In [6]:
# # query to fetch the required data
# df_apps = q(""" 

# with apps_data as (
# select distinct a.application_id,
# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status
# from prod_db.data.applications a
# where date(application_complete_datetime) between '2021-01-01' and '2023-12-31' and business_type ilike '%sole%'
# )

# ,alloy_persons_data_tmp as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# rank() over(partition by d.APPLICATION_ID order by d.APPLICATION_VERSION_ID desc) as ranking
# from prod_db.data.ALLOY_EVALUATIONS_PERSONS d
# inner join apps_data b on d.application_id=b.application_id
# ),


# alloy_persons_data_tmp2 as (
# select * from alloy_persons_data_tmp where ranking=1
# ),

# alloy_persons_data as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# first_value(d.socure_sigma) over(partition by d.application_id order by d.socure_sigma desc) as socure_sigma
# from alloy_persons_data_tmp2 d
# )

# select 
# d.application_id,
# d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,

# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status, e.fraud_score, e.deposit_score

# from alloy_persons_data d 
# inner join apps_data a on d.application_id=a.application_id
# left join prod_db.models.onboarding_model_results e on d.application_id=e.application_id

#     """)

In [7]:
# # save the dataset
# file = 'apps_raw_dataset_2021_2023.pkl'
# path = project_path + 'data/'
# df_apps.reset_index(inplace=True)
# df_apps.to_pickle(path+file)

#### 2022 to 2023: All Apps Data

In [1]:
# # query to fetch the required data
# df_apps = q(""" 

# with apps_data as (
# select distinct a.application_id,
# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status
# from prod_db.data.applications a
# where date(application_complete_datetime) between '2022-01-01' and '2023-12-31' and business_type ilike '%sole%'
# )

# ,alloy_persons_data_tmp as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# rank() over(partition by d.APPLICATION_ID order by d.APPLICATION_VERSION_ID desc) as ranking
# from prod_db.data.ALLOY_EVALUATIONS_PERSONS d
# inner join apps_data b on d.application_id=b.application_id
# ),


# alloy_persons_data_tmp2 as (
# select * from alloy_persons_data_tmp where ranking=1
# ),

# alloy_persons_data as (
# select distinct
# d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
# first_value(d.socure_sigma) over(partition by d.application_id order by d.socure_sigma desc) as socure_sigma
# from alloy_persons_data_tmp2 d
# )

# select 
# d.application_id,
# d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
# d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
# d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
# d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
# d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
# d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
# d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,

# a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
# a.email, a.email_domain, a.business_pitch,
# a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
# a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
# a.number_of_employees, a.purpose_of_account, a.has_international_business,
# a.industry_name, a.industry_category_name, a.website, 
# a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
# a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
# a.company_name, a.phone, a.device, a.flow, a.status, e.fraud_score, e.deposit_score

# from alloy_persons_data d 
# inner join apps_data a on d.application_id=a.application_id
# left join prod_db.models.onboarding_model_results e on d.application_id=e.application_id

#     """)


NameError: name 'q' is not defined

In [21]:
# # save the dataset
# file = 'apps_raw_dataset_2022_2023.pkl'
# path = project_path + 'data/'
# df_apps.reset_index(inplace=True)
# df_apps.to_pickle(path+file)

### OOT Data

In [41]:
# query to fetch the required data, variables to change are the from_date, to_date, od_amount, od_count and days_on_platform
df_oot = q(""" 

-- Data extracting query
with all_businesses as (
select a.application_id, business_id, expensed_fraud_loss, business_type,
case when closure_reason ilike '%system closeout%' then 'System Closeout' else closure_reason end as closure_reason, 
account_create_date, account_close_date, closure_requested_by, closure_is_suspect_fraud,
ACCOUNT_FUNDED_AT, datediff('day', account_create_date, coalesce(ACCOUNT_FUNDED_AT, current_date())) as days_to_fund_account, first_credit_amount,
datediff('day', account_create_date, coalesce(account_close_date, current_date())) as days_to_close_fraud_account,
b.fraud_score, b.deposit_score,
case 
    when b.fraud_score<=0.35 then 1
    when b.fraud_score<=0.6 then 2
    else 3
end as fraud_bin,
case 
    when b.fraud_score<=0.35 then 'low'
    when b.fraud_score<=0.6 then 'moderate'
    else 'high'
end as fraud_category
from prod_db.data.businesses a
left join prod_db.models.onboarding_model_results b on a.application_id=b.application_id
where 1=1 
and account_create_date between '2024-01-01' and '2024-03-31'
and business_type ilike '%sole%'
)

-- Pulling transactions data
,txns_data as (
select a.application_id, a.business_id, a.business_type, a.account_create_date, a.account_funded_at, a.days_to_close_fraud_account,
a.days_to_fund_account, a.account_close_date, b.transaction_id, b.amount, b.type, b.medium, b.status, b.transaction_date, b.running_balance
from all_businesses a
left join prod_db.data.TRANSACTIONS b on a.business_id=b.business_id
)

-- Extracting OD accounts info and filtering for 90 days on book data
,full_txns_data as (
select business_id, 
sum(case when account_funded_at is not null then 1 else 0 end) as account_funded,
sum(case when status='active' and running_balance < -250 then 1 else 0 end) as overdraft_count
from txns_data 
where 1=1 
and datediff('day', account_create_date, transaction_date) <= 90 
and days_to_close_fraud_account <= 90 
and account_close_date is not null
group by 1
)

-- Filtering OD accounts info using the frequency
,overdraft_apps_data as (
select extract('year', a.account_create_date) as year, count(distinct a.business_id) as closed_apps_by_novo_90_days_overdraft_apps 
from all_businesses a 
inner join full_txns_data b on a.business_id=b.business_id
where b.overdraft_count >= 2
group by 1
)

-- Tagging the final target
,final_target_tmp as (
select distinct extract('year', a.account_create_date) as year, a.business_id from all_businesses a
inner join full_txns_data b on a.business_id=b.business_id where 1=1 and b.overdraft_count >= 2

union

select distinct extract('year', a.account_create_date) as year, a.business_id from all_businesses a 
where 1=1 and a.account_close_date is not null and a.closure_requested_by not ilike 'user' and a.expensed_fraud_loss>0
)

-- Filtering the final target
,final_target_tmp2 as (select year, business_id, rank() over(partition by business_id order by year) as rk from final_target_tmp order by rk)


,final_target_tmp3 as ( 
select a.application_id, a.business_id, 
case when a.business_id=b.business_id then 1 else 0 end as target
from all_businesses a
left join 
(select * from final_target_tmp2 where 1=1 and rk=1) b on a.business_id=b.business_id
)

-- final_target data
,final_target as ( 
select a.application_id, a.business_id, a.target, coalesce(b.expensed_fraud_loss,0) as expensed_fraud_loss
from final_target_tmp3 a
left join 
prod_db.data.businesses b on a.business_id=b.business_id
)

-- select count(*) as cnt from all_businesses
-- where 1=1
-- and ACCOUNT_FUNDED_AT is not null
-- and days_to_fund_account <= 90

,apps_data as (
select distinct a.application_id,
a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
a.email, a.email_domain, a.business_pitch,
a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
a.number_of_employees, a.purpose_of_account, a.has_international_business,
a.industry_name, a.industry_category_name, a.website, 
a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
a.company_name, a.phone, a.device, a.flow
from prod_db.data.applications a
inner join final_target b on a.application_id=b.application_id
)

,alloy_persons_data_tmp as (
select distinct
d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
rank() over(partition by d.APPLICATION_ID order by d.APPLICATION_VERSION_ID desc) as ranking
from prod_db.data.ALLOY_EVALUATIONS_PERSONS d
inner join final_target b on d.application_id=b.application_id
),


alloy_persons_data_tmp2 as (
select * from alloy_persons_data_tmp where ranking=1
),

alloy_persons_data as (
select distinct
d.application_id, d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
d.sentilink_id_theft_score, d.socure_reason_code, d.socure_kyc_field_validations, 
d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,
first_value(d.socure_sigma) over(partition by d.application_id order by d.socure_sigma desc) as socure_sigma
from alloy_persons_data_tmp2 d
)

select 
c.*,
d.person_fraud_score, d.person_kyc_score, d.person_fraud_tags, d.person_kyc_tags,
d.iovation_device_type, d.iovation_device_timezone, d.iovation_device_ip, d.iovation_device_ip_isp,
d.iovation_device_ip_org, d.iovation_device_ip_city, d.iovation_device_ip_region, 
d.sentilink_abuse_score, d.sentilink_first_party_synthetic_score, d.sentilink_third_party_synthetic_score,
d.sentilink_id_theft_score, d.socure_sigma, d.socure_reason_code, d.socure_kyc_field_validations, 
d.socure_kyc_reason_code, d.socure_emailrisk, socure_emailrisk_reason_code, d.socure_phonerisk,
d.socure_phonerisk_reason_code, d.socure_addressrisk, d.socure_addressrisk_reason_code, d.carrier, d.line_type,

a.application_start_datetime, a.application_complete_datetime, a.application_resubmitted_datetime,
a.email, a.email_domain, a.business_pitch,
a.estimated_monthly_revenue, a.incoming_ach_payments, a.check_deposit_amount,
a.incoming_wire_transfer, a.outgoing_ach_and_checks, a.outgoing_wire_transfers,
a.number_of_employees, a.purpose_of_account, a.has_international_business,
a.industry_name, a.industry_category_name, a.website, 
a.business_address_city, a.business_address_state, a.business_address_zip, a.ein_ssn, 
a.touch_point_emails, a.owner_list, a.industry_category_from_pitch, a.industry_supercategory_from_pitch,
a.company_name, a.phone, a.device, a.flow, e.fraud_score, e.deposit_score

from final_target c
inner join alloy_persons_data d on c.application_id=d.application_id
inner join apps_data a on c.application_id=a.application_id
left join prod_db.models.onboarding_model_results e on c.application_id=e.application_id

    """)


In [42]:
# save the dataset
file = 'apps_raw_dataset_2024_oot.pkl'
path = project_path + 'data/'
df_oot.reset_index(inplace=True)
df_oot.to_pickle(path+file)